In [13]:
from datetime import datetime
startTime = datetime.now()

l = []
def generateColumns(start, end):
    for i in range(start, end+1):
        l.extend([str(i)+'X', str(i)+'Y'])
    return l

eyes = generateColumns(1, 12)

# reading in the csv as a dataframe
#!pip3 install pandas
import pandas as pd
df = pd.read_csv('EYES.csv')

# selecting the features and target
X = df[eyes]
y = df['truth_value']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 42)

# Data Normalization
from sklearn.preprocessing import StandardScaler as SC
sc = SC()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

import numpy as np
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)

import math
import random

# A class defining the simulated anneaing algorithm
# However, the main implementation is in the Neural Network Algorithm
class SimulatedAnnealing(object):
    def __init__(self,kmax = 100, T0 = 100, tfactor = 0.99):
        self.kmax = kmax
        self.T0 = T0
        self.tfactor = tfactor
        
    
    # Gets a neighbour for the current state    
    def neighbour(self,W):
        rows,columns = W.shape
        new_weights = np.random.randn(rows,columns)
        return new_weights
    
    # Calculates the probability of the new state    
    def prob(self,E,Enew,T):
        val = (E-Enew)/T
        P = math.exp(val)
        return P    
    
    # Returns neighbour to the function that called it    
    def cal(self,W):
            new_weights = self.neighbour(W)
            return new_weights

        
# Class which defines the Neural Network
class NeuralNetwork(object):
    def __init__(self,input_nodes,hidden_nodes,output_nodes):
        #parameters
        self.inputsize = input_nodes
        self.hiddensize = hidden_nodes
        self.outputsize = output_nodes
        self.accuracy = 0
        self.best_accuracy = 0
        
        #Weights
        self.W1 = np.random.randn(self.inputsize, self.hiddensize)
        self.W2 = np.random.randn(self.hiddensize, self.outputsize)
        
    # The forward feed neural network
    def forward(self,x):
        self.z_input_to_hidden = np.dot(x,self.W1)
        self.z_hidden_output = self.relu(self.z_input_to_hidden)
        self.z_hidden_to_output = np.dot(self.z_hidden_output,self.W2)
        self.z_output = self.sigmoid(self.z_hidden_to_output)
        return self.z_output
    
    # The relu activation function    
    def relu(self,val):
        return np.maximum(val,0)
    
    # The sigmoid activation function    
    def sigmoid(self,val):
        return 1/(1+np.exp(-val))
    
    
    # The simulated annealing algorithm implemented by using Simulated Annealing class
    def sim_ann(self,x,y,kmax,T0,tfactor):
        sa = SimulatedAnnealing()
        curr_W = self.W1
        best_W = self.W1
        count = 0
        initial_T0 = T0
        
        # This is to update weights from input layer to hidden layer
        for i in range(0,kmax):
            self.W1 = sa.cal(self.W1)
            o = self.forward(x)
            o = list(o>0.5)
            acc = self.cal_acc(y.shape[0],o,y)
            
            if acc > self.accuracy:
                self.accuracy = acc
                curr_W = self.W1
                if acc > self.best_accuracy:
                    self.best_accuracy = acc
                    best_W = self.W1
            
            else:
                P = sa.prob(self.accuracy,acc,T0)
                if P > random.random():
                    curr_W = self.W1
                    self.accuracy = acc
            T0 = T0 * tfactor
            
        
        # Now, we take the best weights and set them for W1
        self.W1 = best_W
        self.accuracy = self.best_accuracy
        
        curr_W = self.W2
        best_W = self.W2
        count = 0
        T0 = initial_T0
        
        # This is to update the weights from hidden layer to output layer
        for i in range(0,kmax):
            self.W2 = sa.cal(self.W2)
            o = self.forward(x)
            o = list(o>0.5)
            acc = self.cal_acc(y.shape[0],o,y)
            
            if acc > self.accuracy:
                self.accuracy = acc
                curr_W = self.W2
                if acc > self.best_accuracy:
                    self.best_accuracy = acc
                    best_W = self.W2
            
            else:
                P = sa.prob(self.accuracy,acc,T0)
                if P > random.random():
                    curr_W = self.W2
                    self.accuracy = acc
            T0 = T0 * tfactor
            
        # Set the best weights for W2 
        self.W2 = best_W
        self.accuracy = self.best_accuracy
                
        
    # To train the model
    def train(self,x,y):
        o = self.forward(x)
        
        if self.accuracy !=0:
            self.accuracy = self.cal_acc(y.shape[0],o,y)
            self.best_accuracy = self.accuracy
        
        self.sim_ann(x,y,100,1,0.99)
        
    # To test the model on the test data
    def test(self,x):
        o = self.forward(x)
        return o
    
    # To calculate the accuracy of the model
    def cal_acc(self,size,o,y):
        count = 0
        acc = 0
        for j in range(0,size):
            if y[j] == o[j]:
                count += 1
        acc = count/size
        return acc
    
    
# To define number of nodes of every layer    
input_no_nodes = len(X.columns)
hidden_nodes = 8
output_node = 1

# Create instance of our neural network
NN = NeuralNetwork(input_no_nodes,hidden_nodes,output_node)

# Pass training data through neural network 100 times
epochs = 100
for i in range(0,epochs):
    NN.train(X_train,y_train)
    

print("Accuracy of training is:",NN.accuracy)

# Test our data over neural network
y_pred = NN.test(X_test)
y_pred_labels = list(y_pred>0.5)

# Find predicted values
for i in range(len(y_pred_labels)):
    if int(y_pred_labels[i]) == 1:
        y_pred_labels[i] = 1
    else:
        y_pred_labels[i] = 0
        
# Accuracy of model for test data     
acc = NN.cal_acc(y_test.shape[0],y_pred_labels,y_test)

# Putting results of model over test data in data frame
df_results = pd.DataFrame()
df_results['Actual label'] = y_test
df_results['Predicted value'] = y_pred
df_results['Predicted label'] = y_pred_labels
df_results.to_csv('Results_Only_Simulated_Annealing.csv')

print("Accuracy of testing is:",acc)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("\n")
print("Confusion Matrix : ")
print(cm)
print("\n")



# To see how long the model takes to execute
print("\n")
print("Execution time in seconds = ", datetime.now() - startTime)

Accuracy of training is: 0.9133333333333333
Accuracy of testing is: 0.96


Confusion Matrix : 
[[ 8  2]
 [ 0 40]]




Execution time in seconds =  0:00:25.302744
